In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd

/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [177]:
# Test
# Test 2
# Test 3

URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [199]:
# Club International Cups
club_int_comps = []

for j in soup.find_all('tbody')[0].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            club_int_comps.append(j.find('a')['href'])

club_int_comps

['/en/comps/14/history/Copa-Libertadores-Seasons',
 '/en/comps/205/history/Copa-Sudamericana-Seasons',
 '/en/comps/8/history/UEFA-Champions-League-Seasons',
 '/en/comps/19/history/UEFA-Europa-League-Seasons']

In [200]:
# National Team Competitions
national_comps = []

for j in soup.find_all('tbody')[1].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            national_comps.append(j.find('a')['href'])

national_comps

['/en/comps/1/history/World-Cup-Seasons',
 '/en/comps/656/history/Africa-Cup-of-Nations-Seasons',
 '/en/comps/681/history/CONCACAF-Gold-Cup-Seasons',
 '/en/comps/257/history/OFC-Nations-Cup-Seasons',
 '/en/comps/676/history/European-Championship-Seasons',
 '/en/comps/664/history/AFC-Asian-Cup-Seasons']

In [205]:
# Domestic Tier 1 Comps
dom_tier1_comps = []

for j in soup.find_all('tbody')[3].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        dom_tier1_comps.append(j.find('a')['href'])

dom_tier1_comps

'/en/comps/105/history/Liga-FUTVE-Seasons'

In [207]:
all_comps = club_int_comps + national_comps + dom_tier1_comps

len(all_comps)

58

In [213]:
all_comps[-1]

'/en/comps/105/history/Liga-FUTVE-Seasons'

In [214]:
def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [215]:
all_seasons = []

for i in all_comps:
    league_seasons = get_all_seasons(i)
    all_seasons += league_seasons

In [265]:
# all_seasons

In [220]:
# def get_all_player_urls(url_list):
    
#     final_player_urls = []

#     for i in url_list:
#         URL_test = 'https://fbref.com/'+ i
#         page = requests.get(URL_test)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         td1 = soup.find(lambda t: t.text.strip()=='Standard Stats')
#         td2 = td1.find_next('a')
#         url_to_add = td2['href']
#         final_player_urls.append(url_to_add)

#     return final_player_urls

In [240]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [242]:
final_stats = get_all_player_urls(all_seasons)

In [243]:
# URL = 'https://fbref.com/' + final_stats[0]
# page = requests.get(URL)
# soup = BeautifulSoup(page.content, 'html.parser')

In [264]:
# soup.find_all('td', {'data-stat':'player'})

[]